## Production Simulation

In [1]:
# columns = ['label', 'af_center', 'afk_address', 'afk_amount', 'afk_average', 'afk_base', 'afk_bonus', 'afk_borrower', 'afk_city', 'afk_commissions',
#  'afk_continuance', 'afk_continued', 'afk_current', 'afk_date', 'afk_description', 'afk_docusign', 'afk_employee', 'afk_employer', 'afk_employment', 'afk_envelope',
#  'afk_first', 'afk_full', 'afk_grade', 'afk_gross', 'afk_hire', 'afk_hours', 'afk_income', 'afk_increase', 'afk_indicator', 'afk_information',
#  'afk_job', 'afk_last', 'afk_lender', 'afk_line', 'afk_middle', 'afk_military', 'afk_name', 'afk_number', 'afk_original', 'afk_overtime',
#  'afk_page', 'afk_past', 'afk_pay', 'afk_per', 'afk_period', 'afk_phone', 'afk_present', 'afk_probability', 'afk_projected', 'afk_recent',
#  'afk_remarks', 'afk_signature', 'afk_ssn', 'afk_start', 'afk_state', 'afk_status', 'afk_suffix', 'afk_text', 'afk_thru', 'afk_time',
#  'afk_title', 'afk_total', 'afk_type', 'afk_value', 'afk_verification', 'afk_verified', 'afk_week', 'afk_year', 'afk_ytd', 'afk_zipcode',
#  'date_diff', 'day_of_month', 'day_of_week', 'df_center', 'dfk_address', 'dfk_amount', 'dfk_average', 'dfk_base', 'dfk_bonus', 'dfk_borrower',
#  'dfk_city', 'dfk_commissions', 'dfk_continuance', 'dfk_continued', 'dfk_current', 'dfk_date', 'dfk_description', 'dfk_docusign', 'dfk_employee', 'dfk_employer',
#  'dfk_employment', 'dfk_envelope', 'dfk_first', 'dfk_full', 'dfk_grade', 'dfk_gross', 'dfk_hire', 'dfk_hours', 'dfk_income', 'dfk_increase',
#  'dfk_indicator', 'dfk_information', 'dfk_job', 'dfk_last', 'dfk_lender', 'dfk_line', 'dfk_middle', 'dfk_military', 'dfk_name', 'dfk_number',
#  'dfk_original', 'dfk_overtime', 'dfk_page', 'dfk_past', 'dfk_pay', 'dfk_per', 'dfk_period', 'dfk_phone', 'dfk_present', 'dfk_probability',
#  'dfk_projected', 'dfk_recent', 'dfk_remarks', 'dfk_signature', 'dfk_ssn', 'dfk_start', 'dfk_state', 'dfk_status', 'dfk_suffix', 'dfk_text',
#  'dfk_thru', 'dfk_time', 'dfk_title', 'dfk_total', 'dfk_type', 'dfk_value', 'dfk_verification', 'dfk_verified', 'dfk_week', 'dfk_year',
#  'dfk_ytd', 'dfk_zipcode', 'doctype', 'entity_type', 'page_number', 'rid']

In [16]:
import json
import os
import boto3
import tarfile
import pickle as pkl
import service as inference
import entity_selector as selector
import pandas as pd
import xgboost as xgb
import traceback
import numpy as np
import traceback

file_date = '08_25'
# timestamp = '20200812'
timestamp = '08_25'
date_str = timestamp
output_dir = 'argmax_comparison'
#account = "usbank"

# Get config maps
with open('./config/config_james0825.json') as f:
    config = json.load(f)
    
    

In [17]:
def initiate_needed_ets(doctype):
    
    all_dps = list(config['datapoint_map_by_doctype'][doctype].keys())

    lst = []
    for k,l in config['entity_type_to_datapoints'].items():
        for dp in l:
            lst.append((k, dp))

    needed_ets = []
    for dp in all_dps:
        f = [tup for tup in lst if tup[1] == dp]
        if len(f):
            needed_ets.append(f[0][0])

    needed_ets = set(needed_ets)
    return needed_ets

In [18]:
training_results = [{'doctype': 'DEMOGRAPHIC ADDENDUM',
  'status': 'Completed',
  'best_train_jobname': 'xgboost-demographicaddendum-2020-08-31-18-23-57',
  'model_date': '2020-08-31-18-23-57',
  'model_loc': 'https://s3-us-east-1.amazonaws.com/hw-app-aiva-prod/EntityTrainingModel/2020-08-31-18-23-57/demographicaddendum/xgboost-demographicaddendum-2020-08-31-18-23-57/output/model.tar.gz'},
 {'doctype': 'CLOSING DISCLOSURE',
  'status': 'Completed',
  'best_train_jobname': 'xgboost-closingdisclosure-2020-08-31-18-24-04',
  'model_date': '2020-08-31-18-24-04',
  'model_loc': 'https://s3-us-east-1.amazonaws.com/hw-app-aiva-prod/EntityTrainingModel/2020-08-31-18-24-04/closingdisclosure/xgboost-closingdisclosure-2020-08-31-18-24-04/output/model.tar.gz'},
 {'doctype': 'UNDERWRITING TRANSMITTAL - 1008',
  'status': 'Completed',
  'best_train_jobname': 'xgboost-underwritingtransmittal-1008-2020-08-31-18-25-13',
  'model_date': '2020-08-31-18-25-13',
  'model_loc': 'https://s3-us-east-1.amazonaws.com/hw-app-aiva-prod/EntityTrainingModel/2020-08-31-18-25-13/underwritingtransmittal-1008/xgboost-underwritingtransmittal-1008-2020-08-31-18-25-13/output/model.tar.gz'},
 {'doctype': '1003',
  'status': 'Completed',
  'best_train_jobname': 'xgboost-1003-2020-08-31-18-25-45',
  'model_date': '2020-08-31-18-25-45',
  'model_loc': 'https://s3-us-east-1.amazonaws.com/hw-app-aiva-prod/EntityTrainingModel/2020-08-31-18-25-45/1003/xgboost-1003-2020-08-31-18-25-45/output/model.tar.gz'},
 {'doctype': 'PAY STUB',
  'status': 'Completed',
  'best_train_jobname': 'xgboost-paystub-2020-08-31-18-26-55',
  'model_date': '2020-08-31-18-26-55',
  'model_loc': 'https://s3-us-east-1.amazonaws.com/hw-app-aiva-prod/EntityTrainingModel/2020-08-31-18-26-55/paystub/xgboost-paystub-2020-08-31-18-26-55/output/model.tar.gz'},
 {'doctype': 'VERIFICATION OF EMPLOYMENT - FNMA 1005',
  'status': 'Completed',
  'best_train_jobname': 'xgboost-voe-2020-08-31-18-27-22',
  'model_date': '2020-08-31-18-27-22',
  'model_loc': 'https://s3-us-east-1.amazonaws.com/hw-app-aiva-prod/EntityTrainingModel/2020-08-31-18-27-22/verificationofemployment-fnma1005/xgboost-voe-2020-08-31-18-27-22/output/model.tar.gz'},
#  {'doctype': 'VERIFICATION OF ASSET - ACCOUNTCHEK',
#   'status': 'Completed',
#   'best_train_jobname': 'xgboost-voa-2020-08-31-18-27-25',
#   'model_date': '2020-08-31-18-27-25',
#   'model_loc': 'https://s3-us-east-1.amazonaws.com/hw-app-aiva-prod/EntityTrainingModel/2020-08-31-18-27-25/verificationofasset-accountchek/xgboost-voa-2020-08-31-18-27-25/output/model.tar.gz'},
 {'doctype': 'PAYOFF STATEMENT',
  'status': 'Completed',
  'best_train_jobname': 'xgboost-payoffstatement-2020-08-31-18-27-32',
  'model_date': '2020-08-31-18-27-32',
  'model_loc': 'https://s3-us-east-1.amazonaws.com/hw-app-aiva-prod/EntityTrainingModel/2020-08-31-18-27-32/payoffstatement/xgboost-payoffstatement-2020-08-31-18-27-32/output/model.tar.gz'},
 {'doctype': 'FORM W-2',
  'status': 'Completed',
  'best_train_jobname': 'xgboost-formw-2-2020-08-31-18-28-05',
  'model_date': '2020-08-31-18-28-05',
  'model_loc': 'https://s3-us-east-1.amazonaws.com/hw-app-aiva-prod/EntityTrainingModel/2020-08-31-18-28-05/formw-2/xgboost-formw-2-2020-08-31-18-28-05/output/model.tar.gz'},
 {'doctype': 'CREDIT SUPPLEMENT',
  'status': 'Completed',
  'best_train_jobname': 'xgboost-creditsupplement-2020-08-31-18-28-35',
  'model_date': '2020-08-31-18-28-35',
  'model_loc': 'https://s3-us-east-1.amazonaws.com/hw-app-aiva-prod/EntityTrainingModel/2020-08-31-18-28-35/creditsupplement/xgboost-creditsupplement-2020-08-31-18-28-35/output/model.tar.gz'},
 {'doctype': 'VERIFICATION OF EMPLOYMENT - CORRESPONDENCE',
  'status': 'Completed',
  'best_train_jobname': 'xgboost-voe-2020-08-31-18-28-40',
  'model_date': '2020-08-31-18-28-40',
  'model_loc': 'https://s3-us-east-1.amazonaws.com/hw-app-aiva-prod/EntityTrainingModel/2020-08-31-18-28-40/verificationofemployment-correspondence/xgboost-voe-2020-08-31-18-28-40/output/model.tar.gz'},
 {'doctype': 'MORTGAGE STATEMENT',
  'status': 'Completed',
  'best_train_jobname': 'xgboost-mortgagestatement-2020-08-31-18-28-41',
  'model_date': '2020-08-31-18-28-41',
  'model_loc': 'https://s3-us-east-1.amazonaws.com/hw-app-aiva-prod/EntityTrainingModel/2020-08-31-18-28-41/mortgagestatement/xgboost-mortgagestatement-2020-08-31-18-28-41/output/model.tar.gz'},
 {'doctype': 'PROPERTY INSURANCE POLICY - HAZARD/HOMEOWNERS',
  'status': 'Completed',
  'best_train_jobname': 'xgboost-pip-2020-08-31-18-28-46',
  'model_date': '2020-08-31-18-28-46',
  'model_loc': 'https://s3-us-east-1.amazonaws.com/hw-app-aiva-prod/EntityTrainingModel/2020-08-31-18-28-46/propertyinsurancepolicy-hazardhomeowners/xgboost-pip-2020-08-31-18-28-46/output/model.tar.gz'},
#  {'doctype': 'UNDISCLOSED DEBT MONITORING',
#   'status': 'Completed',
#   'best_train_jobname': 'xgboost-undiscloseddebtmonitoring-2020-08-31-18-29-07',
#   'model_date': '2020-08-31-18-29-07',
#   'model_loc': 'https://s3-us-east-1.amazonaws.com/hw-app-aiva-prod/EntityTrainingModel/2020-08-31-18-29-07/undiscloseddebtmonitoring/xgboost-undiscloseddebtmonitoring-2020-08-31-18-29-07/output/model.tar.gz'},
#  {'doctype': 'TITLE COMMITMENT',
#   'status': 'Completed',
#   'best_train_jobname': 'xgboost-titlecommitment-2020-08-31-18-29-12',
#   'model_date': '2020-08-31-18-29-12',
#   'model_loc': 'https://s3-us-east-1.amazonaws.com/hw-app-aiva-prod/EntityTrainingModel/2020-08-31-18-29-12/titlecommitment/xgboost-titlecommitment-2020-08-31-18-29-12/output/model.tar.gz'},
 {'doctype': 'VERIFICATION OF EMPLOYMENT - EQUIFAX',
  'status': 'Completed',
  'best_train_jobname': 'xgboost-voe-2020-08-31-18-29-15',
  'model_date': '2020-08-31-18-29-15',
  'model_loc': 'https://s3-us-east-1.amazonaws.com/hw-app-aiva-prod/EntityTrainingModel/2020-08-31-18-29-15/verificationofemployment-equifax/xgboost-voe-2020-08-31-18-29-15/output/model.tar.gz'},
 {'doctype': 'EARNEST MONEY RECEIPT',
  'status': 'Completed',
  'best_train_jobname': 'xgboost-earnestmoneyreceipt-2020-08-31-18-29-20',
  'model_date': '2020-08-31-18-29-20',
  'model_loc': 'https://s3-us-east-1.amazonaws.com/hw-app-aiva-prod/EntityTrainingModel/2020-08-31-18-29-20/earnestmoneyreceipt/xgboost-earnestmoneyreceipt-2020-08-31-18-29-20/output/model.tar.gz'},
 {'doctype': 'BANK STATEMENT',
  'status': 'Completed',
  'best_train_jobname': 'xgboost-bankstatement-2020-08-31-18-29-20',
  'model_date': '2020-08-31-18-29-20',
  'model_loc': 'https://s3-us-east-1.amazonaws.com/hw-app-aiva-prod/EntityTrainingModel/2020-08-31-18-29-20/bankstatement/xgboost-bankstatement-2020-08-31-18-29-20/output/model.tar.gz'},
 {'doctype': 'VERIFICATION OF EMPLOYMENT FORM OTHER',
  'status': 'Completed',
  'best_train_jobname': 'xgboost-voe-2020-08-31-18-30-32',
  'model_date': '2020-08-31-18-30-32',
  'model_loc': 'https://s3-us-east-1.amazonaws.com/hw-app-aiva-prod/EntityTrainingModel/2020-08-31-18-30-32/verificationofemploymentformother/xgboost-voe-2020-08-31-18-30-32/output/model.tar.gz'}]

In [19]:
def fetch_model(model_loc):
    #Fetch Target Model for the Doctype specified
    model_file_path=model_loc.rsplit('hw-app-aiva-prod/')[1]
    print(model_loc)
    print(model_file_path)
    s3_client = boto3.client('s3')
    s3_client.get_object
    s3_client.download_file(
        'hw-app-aiva-prod', 
        model_file_path,
        'model.tar.gz')

    tar = tarfile.open("model.tar.gz")
    tar.extractall()
    tar.close()

    model = pkl.load(open('xgboost-model', 'rb'))
    return model


In [20]:
# print(training_results[0]['model_loc'])
# model_loc =training_results[0]['model_loc']
# model = fetch_model(model_loc)

In [21]:
import boto3
import io
s3 = boto3.resource('s3')


def fetch_pipeline_set(date_str, dt_slug, account):
    print("I am in getting pipeline json for: ",account)
    path = f'PipelineDataSets/{account}/{dt_slug}_{date_str}_pipeline_set.json'
    print(path)
    obj = s3.Object('hw-app-aiva-prod', path)
    data = io.BytesIO()
    obj.download_fileobj(data)
    val = data.getvalue().decode('utf-8')
    return val
    
def uniquing(doctypes: list):
    return list(set(doctypes))

In [22]:

all_docs=[
    {'account':'usbank',
       'doctypes': uniquing(
           [
               "UNDERWRITING TRANSMITTAL - 1008", 
               "CLOSING DISCLOSURE",
               "DEMOGRAPHIC ADDENDUM",
               "1003"    
            ]
       ) 
    },
    {'account':'caliber',
   'doctypes': uniquing(
       [
           "FORM W-2",
           "PAYOFF STATEMENT", 
           "VERIFICATION OF EMPLOYMENT - FNMA 1005",
           "VERIFICATION OF EMPLOYMENT - EQUIFAX",
           "CREDIT SUPPLEMENT",
           "BANK STATEMENT",
           "PAY STUB",
           "VERIFICATION OF ASSET - ACCOUNTCHEK",
           "VERIFICATION OF EMPLOYMENT FORM OTHER",
           "VERIFICATION OF EMPLOYMENT - CORRESPONDENCE",
           "EARNEST MONEY RECEIPT",
           "TITLE COMMITMENT",
           "UNDISCLOSED DEBT MONITORING",
           'PROPERTY INSURANCE POLICY - HAZARD/HOMEOWNERS',
           "MORTGAGE STATEMENT"
           
        
        ]
       ) 
    }
]



In [23]:
dt_to_account = {}

for dct in all_docs:
    for dt in dct['doctypes']:
        dt_to_account[dt] = dct['account']

dt_to_account

{'CLOSING DISCLOSURE': 'usbank',
 '1003': 'usbank',
 'DEMOGRAPHIC ADDENDUM': 'usbank',
 'UNDERWRITING TRANSMITTAL - 1008': 'usbank',
 'VERIFICATION OF EMPLOYMENT - CORRESPONDENCE': 'caliber',
 'BANK STATEMENT': 'caliber',
 'PAY STUB': 'caliber',
 'UNDISCLOSED DEBT MONITORING': 'caliber',
 'PROPERTY INSURANCE POLICY - HAZARD/HOMEOWNERS': 'caliber',
 'VERIFICATION OF EMPLOYMENT FORM OTHER': 'caliber',
 'VERIFICATION OF EMPLOYMENT - EQUIFAX': 'caliber',
 'PAYOFF STATEMENT': 'caliber',
 'VERIFICATION OF EMPLOYMENT - FNMA 1005': 'caliber',
 'MORTGAGE STATEMENT': 'caliber',
 'FORM W-2': 'caliber',
 'EARNEST MONEY RECEIPT': 'caliber',
 'CREDIT SUPPLEMENT': 'caliber',
 'TITLE COMMITMENT': 'caliber',
 'VERIFICATION OF ASSET - ACCOUNTCHEK': 'caliber'}

In [24]:
for dct in training_results:
    if dct['doctype'] in dt_to_account:
        dct['account'] = dt_to_account[dct['doctype']]
training_results

[{'doctype': 'DEMOGRAPHIC ADDENDUM',
  'status': 'Completed',
  'best_train_jobname': 'xgboost-demographicaddendum-2020-08-31-18-23-57',
  'model_date': '2020-08-31-18-23-57',
  'model_loc': 'https://s3-us-east-1.amazonaws.com/hw-app-aiva-prod/EntityTrainingModel/2020-08-31-18-23-57/demographicaddendum/xgboost-demographicaddendum-2020-08-31-18-23-57/output/model.tar.gz',
  'account': 'usbank'},
 {'doctype': 'CLOSING DISCLOSURE',
  'status': 'Completed',
  'best_train_jobname': 'xgboost-closingdisclosure-2020-08-31-18-24-04',
  'model_date': '2020-08-31-18-24-04',
  'model_loc': 'https://s3-us-east-1.amazonaws.com/hw-app-aiva-prod/EntityTrainingModel/2020-08-31-18-24-04/closingdisclosure/xgboost-closingdisclosure-2020-08-31-18-24-04/output/model.tar.gz',
  'account': 'usbank'},
 {'doctype': 'UNDERWRITING TRANSMITTAL - 1008',
  'status': 'Completed',
  'best_train_jobname': 'xgboost-underwritingtransmittal-1008-2020-08-31-18-25-13',
  'model_date': '2020-08-31-18-25-13',
  'model_loc': '

In [25]:
def inverse_dict_list(dct):
    inverse_dct = defaultdict(list)
    for k,v in dct.items():
        inverse_dct[v].append(k)
    return inverse_dct

def get_dtslug(doctype: str):
    return doctype.replace(' ', '').replace('/','').lower()


def entity_type_subber(entity_type,doctype):
    if entity_type == 'SPECIAL NUMBER':
        return 'CURRENCY'
    elif entity_type not in initiate_needed_ets(doctype):
        return 'OTHER'
    return entity_type

def make_features_df(rid_features, columns,doctype):
    df = pd.DataFrame(rid_features,columns=columns)

    df.loc[:, 'entity_type'] = df.entity_type.apply(entity_type_subber,doctype=doctype)        
    df = df.replace({"entity_type": config["entity_map"]})     

    df = df.drop(
        [
            'entity_id', 'eventtype', 'modification_method', 'remember_id', 'word_id', 'content', 'text',
            'day_of_week', 'date_diff', 'day_of_month', 'rid', 'label', 'datapoint'
        ],
        axis=1,
        errors='ignore'
    )
    df = df.astype('float32')
    return df
    

    
def get_result_row(op_result, indices, matrix, rid_features,doctype,rid):
    op_result_label = op_result['label']

    op_result_index = indices.get(op_result_label, -1)
    predicted_af_center = None
    predicted_df_center = None
    
    predicted_af_center = None
    predicted_df_center = None

    if op_result_index is None or op_result_index < 0:
        predicted_entity_type = None
        predicted_word_ids = None
        predicted_page = None
    else:
        predicted_entity_type = matrix.loc[op_result_index, 'entity_type']#.get('entity_type')                
        predicted_word_ids_str = matrix.loc[op_result_index, 'text']#.get('datapoint', '')
        predicted_word_ids = predicted_word_ids_str.split('#')
        predicted_page = matrix.loc[op_result_index, 'page']#.get('page_number')
        predicted_fes = [
            fe for fe in rid_features 
            if len(set(fe.get('datapoint', '').split('#')).intersection(set(predicted_word_ids))) > 0]

        if len(predicted_fes) > 0:
            predicted_af_center = predicted_fes[0]['af_center']
            predicted_df_center = predicted_fes[0]['df_center']

    closest_fes = [
        fe for fe in rid_features 
        if len(set(fe.get('datapoint', '').split('#')).intersection(set(op_result['word_ids']))) > 0]

    if len(closest_fes) > 0:
        closest_fe = closest_fes[0]
        closest_match_words = closest_fe.get('datapoint', '').split('#')
        closest_match_entity_id = closest_fe.get('entity_id')
        closest_match_entity_type = closest_fe.get('entity_type')
        closest_match_page = closest_fe.get('page_number')
        closest_af_center = closest_fe.get('af_center')
        closest_df_center = closest_fe.get('df_center')
    else:
        closest_match_words = None
        closest_match_entity_id = None
        closest_match_entity_type = None
        closest_match_page = None
        closest_af_center = None
        closest_df_center = None

    result_row = {
        'doctype': doctype,
        'rid': rid,
        'entity_type': op_result['entity_type'],
        'label': op_result['label'],
        'entity_id': op_result['entity_id'],
        'source': op_result['source'],
        'word_ids': op_result['word_ids'],
        'predicted_feature_index': indices.get(op_result['label']),

        'predicted_word_ids': predicted_word_ids,
        'predicted_entity_type': predicted_entity_type,
        'predicted_page': predicted_page,

        'closest_match_entity_type': closest_match_entity_type,
        'closest_match_entity_id': closest_match_entity_id,
        'closest_match_words': closest_match_words,
        'closest_match_page': closest_match_page,
        'predicted_af_center': predicted_af_center,
        'predicted_df_center': predicted_df_center,
        'closest_af_center': closest_af_center,
        'closest_df_center': closest_df_center,

    }

    if predicted_word_ids is None:
        result_row['totally_correct'] = 0
        result_row['partially_correct'] = 0

    else:
        result_row['totally_correct'] = 1 if set(result_row['word_ids']) == set(result_row['predicted_word_ids']) else 0
        result_row['partially_correct'] = 1 if len(set(result_row['word_ids']).intersection(set(result_row['predicted_word_ids']))) > 0  else 0
        
    return result_row

def has_intersection(lsta, lstb):
    return len(set(lsta).intersection(set(lstb))) > 0


def get_prediction_row(label, pred_index, corrected_op_results, matrix, rid_features):
    word_ids = matrix.loc[pred_index,'text'].split('#')
    matches = [op_result['label'] for op_result in corrected_op_results if has_intersection(word_ids, op_result['word_ids'])]
    prediction = 0
    if len(matches) > 0 and label in matches:
        prediction = 1

    return {
        'label': label,
        'pred_index': pred_index,
        'prediction': prediction
    }

In [26]:
def get_the_results(training_result,test_s3_path):
    
    test_df = pd.read_csv(test_s3_path).drop(['Unnamed: 0'], axis=1, errors='ignore')   
    results = []
    prediction_results = []
    result = training_result
    doctype = result['doctype']
    account = training_result['account']
    dt_slug = get_dtslug(doctype)
    
    features = json.loads(fetch_pipeline_set(date_str, dt_slug, account))


#     with open(training_result['pipeline_loc']) as f:
#         features = json.load(f)

    cdModelFlag = "disable"
    os.environ['CDModelFlag'] = cdModelFlag
    inverse_datapoint_map = config['inverse_datapoint_map_by_doctype'][doctype]
    datapoint_map = config['datapoint_map_by_doctype'][doctype]

    model = fetch_model(result['model_loc'])  
    rids = list(features.keys())
    
    print(date_str, dt_slug, account)
    print(len(features))
    print('len rids', len(rids))

    for rid in rids:
        rid_features = features[rid]['vectors']['features']
        metadata = list(map(inference.extractMetadata, rid_features))
        columns = list(test_df.columns) + ['datapoint']
        df = make_features_df(rid_features, columns,doctype)        

        dbm = xgb.DMatrix(df)
        dbm.feature_names = model.feature_names
        pred = model.predict(dbm) # elastic inference. (.ai elastic inference)
        
        pred = model.predict(dbm) # elastic inference. (.ai elastic inference)
#         print('pred_shape', pred.shape)
        if pred.shape[0] == 0:
#             print('skipping a zero')
            continue
        ## Pass prediction softscores to create Inference Output
        inferenceOutput = inference.combineDataMetadata(pred, metadata, doctype)
        predictions = inferenceOutput['body']['predictions']
        body = inferenceOutput['body']
        # Entities actually is the entities... so maybe we can do it differently
        req = body
        matrix = selector.create_matrix(req,doctype)
        matrix = selector.filter_doctype(matrix, doctype)  

#         if pred.shape[0] == 0:
#             continue
            
#         inferenceOutput = inference.combineDataMetadata(pred, metadata, doctype)
#         predictions = inferenceOutput['predictions']
#         matrix = selector.create_matrix(predictions,doctype)
#         matrix = selector.filter_doctype(matrix, doctype)    

        indices = selector.select_datapoints(matrix, doctype)
        actual_entity_types = matrix.entity_type
        operator_results = features[rid]['operator_results']
        corrected_op_results = [op_result for op_result in operator_results if op_result['label'] in datapoint_map]

        for op_result in corrected_op_results:
            result_row = get_result_row(op_result, indices, matrix, rid_features,doctype,rid)
            results.append(result_row)
            
        for label, pred_index in indices.items():           
            if pred_index is not None:
                prediction_row = get_prediction_row(label, pred_index, corrected_op_results, matrix, rid_features)
                prediction_results.append(prediction_row)


    results_df = pd.DataFrame(results)
    allowed_lbs = list(config['datapoint_map_by_doctype'][doctype].keys())
    allowed_ets = list(config['entity_type_to_datapoints'].keys())
    ocrdf = results_df[results_df.word_ids.apply(lambda x: len(x) > 0 and x[0] is not None)]
    results_df = ocrdf[(ocrdf.entity_type.isin(allowed_ets)) & (ocrdf.label.isin(allowed_lbs))]
    
    prediction_df = pd.DataFrame(prediction_results, columns = ['label','pred_index', 'prediction'])
    prediction_df = prediction_df[prediction_df.label.isin(allowed_lbs)]
    
    print("Acount:", account)
    print('accuracy: ', results_df.totally_correct.mean())
    print('false positives: ', (1 - prediction_df.prediction.mean()))
    print("Returning now ")
    return account,doctype,results_df.totally_correct.mean(),(1 - prediction_df.prediction.mean()),results_df.partially_correct.mean()


In [27]:
# frame_data = []
# for training_result in training_results:
#         dt_slug = get_dtslug(training_result['doctype'])
#         test_s3_path = f's3://hw-app-aiva-prod/ScrubbedData/test_train_validate/{file_date}/{dt_slug}_test.csv'
#         acc,doc,qursy,fp = get_the_results(training_result,test_s3_path)
#         temp = {"Account": acc,
#                "Doctype": doc,
#                "Accuracy": qursy,
#                "False Positive Rate": fp}
#         frame_data.append(temp)

In [28]:
frame_data = []
    
for training_result in training_results:
    dt_slug = get_dtslug(training_result['doctype'])
    test_s3_path = f's3://hw-app-aiva-prod/ScrubbedData/test_train_validate/{file_date}/{dt_slug}_test.csv'
    try: 
        acc,doc,qursy,fp,pa = get_the_results(training_result,test_s3_path)
        temp = {"Account": acc,
               "Doctype": doc,
               "Accuracy": qursy,
               "False Positive Rate": fp,
               "partially_correct acc": pa}
        frame_data.append(temp)
    except:
        print("Failed on this DOC: ",training_result)
        print("ERROR: ",traceback.format_exc())
        pass


I am in getting pipeline json for:  usbank
PipelineDataSets/usbank/demographicaddendum_08_25_pipeline_set.json
https://s3-us-east-1.amazonaws.com/hw-app-aiva-prod/EntityTrainingModel/2020-08-31-18-23-57/demographicaddendum/xgboost-demographicaddendum-2020-08-31-18-23-57/output/model.tar.gz
EntityTrainingModel/2020-08-31-18-23-57/demographicaddendum/xgboost-demographicaddendum-2020-08-31-18-23-57/output/model.tar.gz
08_25 demographicaddendum usbank
119
len rids 119
Acount: usbank
accuracy:  0.6611295681063123
false positives:  0.5906542056074766
Returning now 
I am in getting pipeline json for:  usbank
PipelineDataSets/usbank/closingdisclosure_08_25_pipeline_set.json
https://s3-us-east-1.amazonaws.com/hw-app-aiva-prod/EntityTrainingModel/2020-08-31-18-24-04/closingdisclosure/xgboost-closingdisclosure-2020-08-31-18-24-04/output/model.tar.gz
EntityTrainingModel/2020-08-31-18-24-04/closingdisclosure/xgboost-closingdisclosure-2020-08-31-18-24-04/output/model.tar.gz
08_25 closingdisclosure u

08_25 bankstatement caliber
425
len rids 425
Acount: caliber
accuracy:  0.19213825821755337
false positives:  0.8503762726870296
Returning now 
I am in getting pipeline json for:  caliber
PipelineDataSets/caliber/verificationofemploymentformother_08_25_pipeline_set.json
https://s3-us-east-1.amazonaws.com/hw-app-aiva-prod/EntityTrainingModel/2020-08-31-18-30-32/verificationofemploymentformother/xgboost-voe-2020-08-31-18-30-32/output/model.tar.gz
EntityTrainingModel/2020-08-31-18-30-32/verificationofemploymentformother/xgboost-voe-2020-08-31-18-30-32/output/model.tar.gz
08_25 verificationofemploymentformother caliber
32
len rids 32
Acount: caliber
accuracy:  0.28112449799196787
false positives:  0.7266483516483517
Returning now 


In [29]:
snorkel_frame = pd.DataFrame(frame_data)
snorkel_frame

,Account,Doctype,Accuracy,False Positive Rate,partially_correct acc
0,usbank,DEMOGRAPHIC ADDENDUM,0.661130,0.590654,0.727575
1,usbank,CLOSING DISCLOSURE,0.529677,0.584388,0.623541
2,usbank,UNDERWRITING TRANSMITTAL - 1008,0.478281,0.397984,0.719290
3,usbank,1003,0.420795,0.667775,0.516623
4,caliber,PAY STUB,0.357595,0.723167,0.445411
5,caliber,VERIFICATION OF EMPLOYMENT - FNMA 1005,0.281163,0.615000,0.419668
6,caliber,PAYOFF STATEMENT,0.354478,0.720164,0.403664
7,caliber,FORM W-2,0.196433,0.756732,0.308779
8,caliber,CREDIT SUPPLEMENT,0.295556,0.756098,0.351111
9,caliber,VERIFICATION OF EMPLOYMENT - CORRESPONDENCE,0.350000,0.860465,0.400000


In [30]:
snorkel_frame.to_excel("normal.xls",index=False)

# Debugging on failures

In [ ]:
bakara = {'doctype': 'CLOSING DISCLOSURE', 'status': 'Completed', 'best_train_jobname': 'xgboost-closingdisclosure-2020-08-31-18-24-04', 'model_date': '2020-08-31-18-24-04', 'model_loc': 'https://s3-us-east-1.amazonaws.com/hw-app-aiva-prod/EntityTrainingModel/2020-08-31-18-24-04/closingdisclosure/xgboost-closingdisclosure-2020-08-31-18-24-04/output/model.tar.gz', 'account': 'usbank'}

In [ ]:
dt_slug = get_dtslug("CLOSING DISCLOSURE")
test_s3_path = f's3://hw-app-aiva-prod/ScrubbedData/test_train_validate/{file_date}/{dt_slug}_test.csv'

acc,doc,qursy,fp = get_the_results(bakara,test_s3_path)
temp = {"Account": acc,
       "Doctype": doc,
       "Accuracy": qursy,
       "False Positive Rate": fp}
frame_data.append(temp)